##Features a agregar

In [ ]:
features = [
  'text' #OK
  'Keyword' #OK 
  'localidad' #OK #limpiar las feas. 
  'longitud tweets' #OK
  'cantidad_apariciones_keyword'#OK #contar cantidad de apariciones de las keywords y asignarle ese numero al tweet que contenga esa keyword
  'cantidad de signos de puntuacion' #OK
  'cantidad de stop-words' #OK
  'signo de pregunta' #OK
  'signo de exclamacion' #OK
  'etiqueta (@)' #OK
  'spelling good_bad' #OK TARDA MUCHO
  'subjetividad' #OK
  'Cantidad de frases en un tweet' #OK
  'Cantidad de hashtags' #OK
  'cantidad de url'#OK
  'cantidad de palabras unicas en el texto' #OK
  'promedio del largo de las palabras en el tweet' #OK
  'cantidad de palabras en el tweet' #OK
]

In [ ]:
import pandas as pd
!pip install nltk 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np
import string
from textblob import TextBlob
from textblob import Word
import statistics

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

In [ ]:
id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


1) Agregamos feature 'longitud de tweets'

In [ ]:
train['longitud tweets'] = train['text'].apply(lambda x: len(x))
test['longitud tweets'] = test['text'].apply(lambda x: len(x))

2) Limpiamos las localidades que aparecen en los set de datos, aquellas que no correspondan a una localidad se la reeemplza por 'no_location'

In [ ]:
def limpieza_localidades(location):

  if (location.find('www.') != -1 or location.find('WWW.') != -1):
    return 'no_location'
  if (location.find(',') != -1 or location.find('.') != -1):
    loc = location.replace(',', '')
    loc = location.replace('.', '')
    if loc.isalpha() and (len(loc) < 100): #valuación del estado del csv a analizar
     return location.upper()
  if location.isalpha() and (len(location) < 100):
    return location.upper()
  return 'no_location'

In [ ]:
def union_usa(df):
  df['location'] = df['location'].replace("U.S.A", "USA")
  df['location'] = df['location'].replace("UNITED STATES", "USA")
  df['location'] = df['location'].replace("U.S", "USA")
  df['location'] = df['location'].replace("US", "USA")
  return 

train:

In [ ]:
train['location'] = train['location'].fillna('no_location')
train['location'] = train['location'].apply(lambda x: limpieza_localidades(x))
union_usa(train)

Test:

In [ ]:
test['location'] = test['location'].fillna('no_location')
test['location'] = test['location'].apply(lambda x: limpieza_localidades(x))
union_usa(test)

3) Reemplazamos Nan de Keyword por 'no_keyword'

In [ ]:
train['keyword'] = train['keyword'].fillna('no_keyword')
test['keyword'] = test['keyword'].fillna('no_keyword')

4) Contamos palabras en cada tweet.

In [ ]:
train['cant_palabras'] = train['text'].apply(lambda x: len(str(x).split()))
test['cant_palabras'] = test['text'].apply(lambda x: len(str(x).split()))

5) Contamos palabras unicas en cada tweet

In [ ]:
train['palabras_unicas'] = train['text'].apply(lambda x: len(set(str(x).split())))
test['palabras_unicas'] = test['text'].apply(lambda x: len(set(str(x).split())))

6) Contamos Stop-Words por tweet.

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
train['cant_stop_words'] = train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))
test['cant_stop_words'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))

7) Contamos cantidad de URLs en cada tweet

In [ ]:
train['cant_url'] = train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w or 'www.' in w]))
test['cant_url'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w or 'www.' in w]))

8) Promedio de longitud de las palabras del tweet.

In [ ]:
train['prom_long_palabra'] = train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test['prom_long_palabra'] = test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

9) Contamos puntuacion en cada tweet

In [ ]:
train['cant_puntuacion'] = train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test['cant_puntuacion'] = test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

10) Contamos hashtags que aparecen en cada tweet.

In [ ]:
train['cant_hashtag'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test['cant_hashtag'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

11) Contamos menciones en cada tweet

In [ ]:
train['cant_mencion'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test['cant_mencion'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

12) Contamos cantidad de signos de pregunta en los tweets.

In [ ]:
train['cant_signos_pregunta'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '?']))
test['cant_signos_pregunta'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '?']))

13) Contamos cantidad de signos de exclamacion

In [ ]:
train['cant_signos_exclamacion'] = train['text'].apply(lambda x: len([c for c in str(x) if c == '!']))
test['cant_signos_exclamacion'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '!']))

14) Nos fijamos la subjetividad que tiene cada tweet.

In [ ]:
train['subjetividad'] = train['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
test['subjetividad'] = test['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

15) Obtenemos un tag si la frase esta bien escrita o no. SpellCheck

In [ ]:
def spell_check(text):
  text_lista = text.split(' ')
  for w in text_lista:
    if Word(w).spellcheck()[0][1] != 1:
      return 0
  return 1

In [ ]:
#train['spellcheck'] = train['text'].apply(lambda x: spell_check(x))
#test['spellcheck'] = test['text'].apply(lambda x: spell_check(x))

16) Contamos cantidad de oraciones en cada tweet.

In [ ]:
train['cant_oraciones'] = train['text'].apply(lambda x: len(sent_tokenize(x)))
test['cant_oraciones'] = test['text'].apply(lambda x: len(sent_tokenize(x)))

17) Contar cantidad de apariciones de las keywords y asignarle ese numero al tweet que contenga esa keyword

In [ ]:
train['cant_apariciones_keyword'] = train.loc[:,['keyword', 'id']].groupby('keyword').transform('count')
test['cant_apariciones_keyword'] = test.loc[:,['keyword', 'id']].groupby('keyword').transform('count')

18) Contamos cantidad de numeros que aparecen en el tweet

In [ ]:
def count_number(text):
    count = 0
    for char in text:
        if char.isdigit():
            count += 1
    return count

train['cant_numeros'] = train['text'].apply(lambda x: count_number(x))
test['cant_numeros'] = test['text'].apply(lambda x: count_number(x))

19) Contamos cantidad de letras minusculas en el tweet

In [ ]:
def count_lower(text): 
    lower = 0
    for i in range(len(text)): 
        if (ord(text[i]) >= 97 and ord(text[i]) <= 122): 
            lower += 1
    return lower

train['cant_minusculas'] = train['text'].apply(lambda x: count_lower(x))
test['cant_minusculas'] = test['text'].apply(lambda x: count_lower(x))

20) Contamos cantidad de letras mayusculas en el tweet

In [ ]:
def count_upper(text):
    upper = 0
    for i in range(len(text)):
        if (ord(text[i]) >= 65 and ord(text[i]) <= 90): 
            upper += 1
    return upper

train['cant_mayusculas'] = train['text'].apply(lambda x: count_upper(x))
test['cant_mayusculas'] = test['text'].apply(lambda x: count_upper(x))

21) Contamos cantidad de vocales en el tweet

In [ ]:
def count_vowel(text):    
    count = 0
    vowel = set("aeiouAEIOU") 
    for char in text: 
        if char in vowel: 
            count = count + 1
    return count

train['cant_vocales'] = train['text'].apply(lambda x: count_vowel(x))
test['cant_vocales'] = test['text'].apply(lambda x: count_vowel(x))

22) Contamos cantidad de consonantes

In [ ]:
def count_consonant(str):    
    count = 0
    consonant = set("bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ") 
    for alphabet in str: 
        if alphabet in consonant: 
            count = count + 1
    return count

train['cant_consonant'] = train['text'].apply(lambda x: count_consonant(x))
test['cant_consonant'] = test['text'].apply(lambda x: count_consonant(x))

Descargamos ambos DF para que persistan.

In [ ]:
from google.colab import files

In [ ]:
train.to_csv('train_features.csv', index=False)
files.download('train_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test.to_csv('test_features.csv', index=False)
files.download('test_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>